# Analyse de l'impact de la Coupe du Monde de Rugby 2023 sur le trafic des transports en commun

## Introduction

Ce notebook a pour objectif d'analyser l'impact de la Coupe du Monde de Rugby 2023 (du 8 septembre au 28 octobre 2023) sur la fréquentation du RER B. Nous utiliserons un modèle SARIMAX (v1 avec FB Prophet) pour prédire le trafic normal attendu et le comparer avec les données réelles pendant cette période.

## 1. Configuration de l'environnement

In [ ]:
# Import des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import holidays
import re
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

# Configuration de l'affichage
warnings.filterwarnings("ignore")
plt.style.use('ggplot')
sns.set(style="whitegrid")
%matplotlib inline
pd.set_option('display.max_columns', None)

## 2. Chargement des données

In [ ]:
# Chargement des données de validations
try:
    df = pd.read_csv('final_data/df_rer_b.csv', parse_dates=['JOUR'])
    print(f"Données chargées : {df.shape[0]} lignes et {df.shape[1]} colonnes")
except Exception as e:
    print(f"Erreur lors du chargement du fichier CSV : {e}")

# Aperçu des données
df.head()

In [ ]:
# Chargement des données des stations/lignes
try:
    df_lignes = pd.read_csv('final_data/df_lignes_ferrees.csv')
    print(f"Données des lignes chargées : {df_lignes.shape[0]} lignes et {df_lignes.shape[1]} colonnes")
except Exception as e:
    print(f"Erreur lors du chargement du fichier des lignes : {e}")

# Aperçu des lignes
df_lignes.head()

## 5. Définition du modèle SARIMAX et fonction de prédiction

In [ ]:
def sarimax_prediction_cdm(arret):
    """
    Fonction pour prédire le trafic normal attendu pendant la Coupe du Monde de Rugby 2023
    et le comparer aux données réelles pour mesurer l'impact de l'événement.
    
    Args:
        arret (str): Nom de l'arrêt/gare à analyser
        
    Returns:
        tuple: DataFrame avec les résultats et MAPE (erreur de prédiction)
    """
    try:
        df = pd.read_csv('final_data/df_rer_b.csv', parse_dates=['JOUR'])
    except Exception as e:
        print(f"Erreur lors du chargement du fichier CSV : {e}")
        return None

    # Nettoyer l'arrêt demandé
    arret_clean = re.sub(r'\s+', '', re.sub(r'\s*\([^)]*\)', '', str(arret)).replace('-', ''))
    df['LIBELLE_ARRET_clean'] = df['LIBELLE_ARRET'].apply(
        lambda x: re.sub(r'\s+', '', re.sub(r'\s*\([^)]*\)', '', str(x)).replace('-', ''))
    )
    df = df[df['LIBELLE_ARRET_clean'] == arret_clean].copy()

    if df.empty:
        print(f"Aucune donnée trouvée pour l'arrêt : {arret}")
        return None

    # Préparation des données
    df = df.rename(columns={'JOUR': 'ds', 'NB_VALD': 'y'})
    df = df[['ds', 'y', 'LIBELLE_ARRET', 'LIBELLE_ARRET_clean', 'IS_VACANCE']].sort_values('ds')
    df['DAY_OF_WEEK'] = df['ds'].dt.weekday / 6
    df['IS_WEEKEND'] = (df['ds'].dt.weekday >= 5).astype(int)

    # Événements
    fr_holidays = holidays.France(years=range(2015, 2025))
    euro_2016_start = pd.Timestamp('2016-06-10')
    euro_2016_end = pd.Timestamp('2016-07-10')
    cdm_rugby_2023_start = pd.Timestamp('2023-09-08')
    cdm_rugby_2023_end = pd.Timestamp('2023-10-28')
    evenement_dates = [
        '2015-05-23', '2015-05-26', '2017-07-15', '2017-07-16', '2017-07-25',
        '2018-07-14', '2019-06-07', '2019-06-08', '2022-05-21', '2022-07-29',
        '2015-11-13', '2017-03-28', '2017-10-07', '2017-10-10', '2018-03-23',
        '2018-06-09', '2019-06-02', '2019-09-07', '2019-09-10', '2020-09-08',
        '2021-06-08', '2021-09-01', '2021-09-04', '2021-09-07', '2022-05-28',
        '2016-02-06', '2017-02-12', '2018-02-03', '2019-02-01', '2020-02-02',
        '2021-02-14', '2022-02-12', '2021-11-20', '2015-06-13', '2016-06-24',
        '2017-06-04', '2018-06-02', '2019-06-15', '2021-06-25', '2022-06-24'
    ]
    evenement_dates = set(pd.to_datetime(evenement_dates))
    
    def is_evenement(date):
        if euro_2016_start <= date <= euro_2016_end:
            return 1
        if cdm_rugby_2023_start <= date <= cdm_rugby_2023_end:
            return 1
        if date in evenement_dates:
            return 1
        return 0

    df['IS_EVENEMENT'] = df['ds'].apply(is_evenement)
    features = ['DAY_OF_WEEK', 'IS_WEEKEND', 'IS_VACANCE', 'IS_EVENEMENT']

    # Log + normalisation
    df['y_log'] = np.log1p(df['y'])
    y_scaler = MinMaxScaler()
    df['y_scaled'] = y_scaler.fit_transform(df[['y_log']])

    # Train / test split
    train_df = df[(df['ds'].dt.year >= 2015) & (df['ds'].dt.year <= 2022)].copy()
    test_df = df[(df['ds'] >= cdm_rugby_2023_start) & (df['ds'] <= cdm_rugby_2023_end)].copy()

    # Exclusion spécifique des 23/09 et 24/09 pour certaines gares travaux
    arrets_exclus = ['AULNAYSOUSBOIS', 'SEVRANLIVRY', 'VERTGALANT', 'VILLEPARISISMITRYLENEUF', 'MITRYCLAYE']
    if df['LIBELLE_ARRET_clean'].iloc[0] in arrets_exclus:
        test_df = test_df[~test_df['ds'].isin([pd.Timestamp('2023-09-23'), pd.Timestamp('2023-09-24')])]

    # Exclusion spécifique du 30/09/2023 pour certains arrêts - travaux
    arrets_exclus_30sept = ['LESBACONNETS', 'MASSYVERRIERES']
    if df['LIBELLE_ARRET_clean'].iloc[0] in arrets_exclus_30sept:
        test_df = test_df[test_df['ds'] != pd.Timestamp('2023-09-30')]

   

    if train_df.dropna().shape[0] < 2 or test_df.dropna().shape[0] < 2:
        print("Pas assez de données pour entraîner ou tester SARIMAX.")
        return None

    # Exogènes
    exog_train = train_df[features]
    exog_test = test_df[features]

    # Entraînement du modèle SARIMAX
    model = SARIMAX(train_df['y_scaled'], order=(2, 1, 2), seasonal_order=(1, 0, 1, 7),
                    exog=exog_train, enforce_stationarity=False, enforce_invertibility=False)
    results = model.fit(disp=False)

    # Indices pour prédiction (utiliser des indices entiers)
    start_idx = len(train_df)
    end_idx = start_idx + len(test_df) - 1

    pred_scaled = results.predict(start=start_idx, end=end_idx, exog=exog_test)
    pred_log = y_scaler.inverse_transform(pred_scaled.values.reshape(-1,1)).flatten()
    pred = np.expm1(pred_log)

    # MAPE
    mape = mean_absolute_percentage_error(test_df['y'], pred) * 100

    # Visualisation
    """
    plt.figure(figsize=(12, 6))
    plt.plot(test_df['ds'], test_df['y'], label='Valeurs réelles', color='blue')
    plt.plot(test_df['ds'], pred, label='Prédictions SARIMAX', color='orange')
    plt.title(f"Prédictions SARIMAX pour 2023 (CDM Rugby) – arrêt {arret}")
    plt.xlabel("Date")
    plt.ylabel("Nombre de validations")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    """

    # Résultat
    df_result = test_df[['ds', 'y']].copy()
    df_result['yhat'] = np.round(pred).astype(int)
    
    return df_result, f"{mape:.2f}%"

## 3. Analyse de l'impact sur le Stade de France

In [ ]:
# Analyse pour la gare du Stade de France
gare = 'LA PLAINE-STADE DE FRANCE'  #RER B
result, mape = sarimax_prediction_cdm(gare)

print(f"MAPE pour {gare}: {mape}")
result.head()

In [ ]:
# Visualisation de l'impact sur la gare du Stade de France
plt.figure(figsize=(14, 7))
plt.plot(result['ds'], result['y'], label='Validations réelles', color='blue', marker='o')
plt.plot(result['ds'], result['yhat'], label='Prédictions (trafic normal)', color='orange', linestyle='--')
plt.fill_between(result['ds'], result['yhat'], result['y'], 
                 where=result['y'] > result['yhat'], 
                 color='green', alpha=0.3, label='Impact positif CDM')
plt.fill_between(result['ds'], result['yhat'], result['y'], 
                 where=result['y'] < result['yhat'], 
                 color='red', alpha=0.3, label='Impact négatif CDM')

# Ajout des dates des matchs au Stade de France
matchs_stade_france = ['2023-09-08', '2023-09-14', '2023-09-23', '2023-10-06', '2023-10-15', '2023-10-20', '2023-10-21', '2023-10-28']
for match in matchs_stade_france:
    plt.axvline(x=pd.Timestamp(match), color='purple', linestyle=':', alpha=0.7)

plt.title(f"Impact de la Coupe du Monde de Rugby 2023 sur la gare {gare}", fontsize=15)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Nombre de validations", fontsize=12)
plt.xticks(rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Prédriction de l'affluence sur l'ensemble du RER B durant la CDM

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
import time

# Charger le fichier CSV
df = pd.read_csv('final_data/sarimax_prediction_rer_b.csv', sep=',', parse_dates=['ds'])
#Ce sont les données finales de prédiction pour toutes les stations du RER B durant la cdm (environ 10min de code)

# On filtre pour éviter les divisions par zéro
df_nonzero = df[df['y'] != 0]

# Calcul du MAPE
mape = mean_absolute_percentage_error(df_nonzero['y'], df_nonzero['yhat']) * 100
print(f"MAPE (scikit-learn) : {mape:.2f}%")


total_yhat = df['yhat'].sum() #yhat = prediction
total_y = df['y'].sum()

print(f"Nombre de validations total prédit pdt la CDM de rugby: {total_yhat:,.0f}".replace(',', ' '))
print(f"Nombre réel de validations pdt la CDM de rugby: {total_y:,.0f}".replace(',', ' '))

map_old_year = {
    2015: 23864311,
    2016: 25201884,
    2017: 26451808,
    2018: 26642013,
    2019: 26631863,
    2021: 23797427
}
#Nous avons calculé ces valeurs à partir de la base de données df_rer_b.csv au préalable
for year, value in map_old_year.items():
    print(f"Nombre de validations total prédit en {year} à la même période que la cdm: {value:,.0f}".replace(',', ' '))


## 5. Conclusion et synthèse

### Principaux résultats de l'analyse

Cette analyse sur la cdm de rubgy 2023 sur le RER B nous a permis:

- D'utiliser les modèles SARIMAX et Prophet pour la prédiction.

- Feature engineering : jour férié, vacance, evenement (sportif, artisituqe)

- L'analyse des données et solution des erreurs/évenement imprévisble ou inconnu.

- L'intégration de contexte événementiel (matchs, concerts) dans le modèle.

- Ouverture : approche réplicable pour d’autres événements (JO 2024, prolongement/ouverture de lignes).